# Video Processing for BT Lab
Functions to check if run is good
<br>
<br>
![UofC logo](./pictures/uofc_logo-black.jpg)

In [27]:
#import libraries
import os
import re
import json
import numpy as np
import pandas as pd
import traceback
from timeit import default_timer as timer
import cv2

In [2]:
# make sure to update path
user_drive = input("Enter user drive: ").upper()
video_path = f"{user_drive}:/Christian/DI_centre_structured"
input(f"Is this the right directory - {video_path}?")

''

In [3]:
VIDEO_CHARACTERISTICS = {
    "With Blankets" : "WB",
    "B" : "WB",
    "Without Blankets" : "WOB",
    "WOB": "WOB",
    "3 Meters" : "3m",
    "2 Meters" : "2m",
    "Hold Breath" : "HB",
    "Hold Breathe" : "HB",
    "H" : "HB",
    "Relaxed" : "rel",
    "R": "rel",
}

In [4]:
# local dirs
repo_dir = os.getcwd()
json_dir = repo_dir + "/records/JSON"
log_dir = repo_dir + "/records/logs"

In [5]:
# creates folder for exported frames if it does not exist """
def set_folder(save_folder:str) -> None:
    if not os.path.isdir(save_folder):
        os.mkdir(save_folder)
    else:
        for files in os.listdir(save_folder):
            os.remove(os.path.join(save_folder, files))

In [6]:
# converts video into frames
def run_video_to_frame(video_path:str , save_folder:str, frame_frequency: dict, new_fps:int, patient_id:str) -> list["str"]:
    # open video file
    video = cv2.VideoCapture(video_path)
    assert video.isOpened()

    # unpack metadata
    vid_fps = int(video.get(cv2.CAP_PROP_FPS))
    number_of_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    vid_length_floored = number_of_frames // vid_fps

    # local variables
    pick_counter, save_counter, set_counter, true_frames = [0, 0, 0, 0]
    list_of_frames = [frame for frame, frequency in frame_frequency.items()]
    expected_frames = new_fps*vid_length_floored
    success = True

    # make folder if it doesn't exist
    set_folder(save_folder)
    
    # start video processing
    start_time  = timer()
    while success and set_counter < vid_length_floored:
        success, frame = video.read()

        if not success:
            break

        if pick_counter in list_of_frames:
            num_of_save = frame_frequency.get(pick_counter)

            for _ in range(num_of_save):
                frame_name = f"{patient_id}_{save_counter}.png"
                frame_path = os.path.join(save_folder, frame_name)
                cv2.imwrite(frame_path, frame)
                save_counter += 1

        # update counters
        pick_counter += 1
        true_frames += 1
        if (pick_counter == vid_fps):
            set_counter += 1
            pick_counter = 0
    
    video.release()
    
    # get timing for one export
    end_time = timer()
    time_delta = end_time - start_time
    print(f"\nDone in {time_delta} seconds.")
    
    if save_counter != expected_frames:
        raise ValueError(f"Expected {expected_frames} frames, but got {save_counter} frames")

    return [set_counter, save_counter, true_frames]

In [7]:
# gets folder paths
def get_frames_path(local_path: str, level:str) -> list["str"]:
    fixed_path = local_path.replace("\\", "/")
    fixed_path_split = fixed_path.split("/")
    video_folder = "/".join(fixed_path_split[:-1])
    video_filename = fixed_path_split[-1].split(".")[0]
    folder_path = video_folder + f"/frames_{video_filename}_{level}"
    return [folder_path, video_folder]

In [8]:
# reads JSON file
def load_json(json_dir:str, filename:str) -> dict:
    full_path = json_dir + "/" + filename

    with open(full_path, "r") as json_data:
        data = json.load(json_data)

    return(data)

In [9]:
# re samples number of frames
def up_sample(old_fps: int, new_fps: int, start:int) -> list[int]:
    frames_arr = np.arange(start, old_fps, dtype=int)
    frames_interp = np.linspace(start, old_fps-1, new_fps)
    nearest_indices = np.round(frames_interp).astype(int)
    up_sampled_list = np.take(frames_arr, nearest_indices, mode='wrap')

    return up_sampled_list.tolist()

In [10]:
# patient_id
def get_id(patient_data: dict, video_count:int) -> str:
    alias = patient_data["alias"]
    blanket = VIDEO_CHARACTERISTICS.get(patient_data["blanket"], "?")
    distance = VIDEO_CHARACTERISTICS.get(patient_data["distance"].title(), "?")
    breathing = VIDEO_CHARACTERISTICS.get(patient_data["breathing"], "?")
    id = alias + "-" + distance + "-" + blanket + "-" + breathing
    id = f"{alias}_{video_count}-{distance}-{blanket}-{breathing}"
    return id

In [11]:
# checks if folders have the correct number ofr frames
def run_folder_check(video_path:str , save_folder:str, frame_frequency: dict, new_fps:int) -> None:
    # open video file
    video = cv2.VideoCapture(video_path)
    assert video.isOpened()

    # unpack metadata
    vid_fps = int(video.get(cv2.CAP_PROP_FPS))
    number_of_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    vid_length_floored = number_of_frames // vid_fps

    # local variables
    expected_frames = int(new_fps*vid_length_floored)
    number_of_files_in_folder = int(len(os.listdir(save_folder)))

    if not os.path.exists(save_folder):
        print(f"{save_folder} does not exists.")
        return save_folder

    video.release()

    if number_of_files_in_folder != expected_frames:
        print(f"{save_folder} - Not enough frames in folder. Only found {number_of_files_in_folder} files. Expecting {expected_frames}.")
        return save_folder

In [12]:
# driver code for checking empty folders
def check_if_empty_folders(all_patients:dict, level: str, new_fps:int, user_drive:str) -> list:
    visited_folders = {}    
    rerun_folders = []
    frames_folders = []

    for json_index, patient_info in all_patients.items():
        try:
            video_path = patient_info["local path"]
            old_fps = int(patient_info["old fps"])

            frames_folder, video_folder = get_frames_path(video_path, level)
            visited_folders[video_folder] = visited_folders.get(video_folder, 0) + 1
            frames_folders.append(frames_folder)
            video_count = visited_folders[video_folder]

            frames_to_pick = up_sample(old_fps, new_fps, 1)
            frames_idx = pd.Index(frames_to_pick, name="frames")
            frame_frequency = frames_idx.value_counts()

            if len(frames_to_pick) != new_fps:
                raise ValueError("Number of frames to pick is not equal to new fps")

            # get id
            patient_id = get_id(patient_info, video_count)

            # run video to frames
            rerun_folder = run_folder_check(video_path, frames_folder, frame_frequency, new_fps)
            
            if rerun_folder != None:
                print("Rerunning folder: ", rerun_folder)
                rerun_folders.append(rerun_folder)
                run_video_to_frame(video_path, frames_folder, frame_frequency, new_fps, patient_id)

        except Exception as e:
            traceback.print_exc()
            print(f'''{type(e)}: {e} for video {patient_info["filename"]}''')

    return [rerun_folders, frames_folders]

# Run tests

In [13]:
""" local vars"""

rgb_fps = {
    "lower_bound": 10,
    "upper_bound": 20
}

thermal_fps = {
    "lower_bound": 5,
    "upper_bound": 10
}

In [14]:
""" load JSON files """

metadata_rgb = load_json(json_dir, "/rgb_complete.json")
metadata_thermal = load_json(json_dir, "/thermal_complete.json")

In [15]:
""" check videos (rgb) """
all_rgb_folders = []
all_rgb_rerun_folders = []

for level, new_fps in rgb_fps.items():
    print(f"\nAdjusting FPS to {new_fps}\n" + "="*50)
    rerun_folders, visited_folders = check_if_empty_folders(metadata_rgb, level, new_fps, user_drive)
    all_rgb_rerun_folders.append(rerun_folders)
    all_rgb_folders.append(visited_folders)


Adjusting FPS to 10

Adjusting FPS to 20


In [16]:
""" check videos (thermal) """
all_thermal_folders = []
all_thermal_rerun_folders = []

for level, new_fps in thermal_fps.items():
    print(f"\nAdjusting FPS to {new_fps}\n" + "="*50)
    rerun_folders, visited_folders = check_if_empty_folders(metadata_thermal, level, new_fps, user_drive)
    all_thermal_folders.append(visited_folders)
    all_thermal_rerun_folders.append(rerun_folders)


Adjusting FPS to 5

Adjusting FPS to 10


# Getting labels

In [46]:
# gets labels based on breathing

def get_label_breathing(visited_folders: list[str]) -> [list, list]:
    all_frames_path = []
    all_labels = []

    hold_breath_pattern = r"/Hold Breath/|/Hold Breathe/|/H/"
    relaxed_pattern = r"/Relaxed/|/R/"

    for visited_folder in visited_folders:
        if re.search(relaxed_pattern, visited_folder):
            all_labels.append(0) # 0 for relaxed pattern
        elif re.search(hold_breath_pattern, visited_folder):
            all_labels.append(1) # 1 for hold breath pattern
        else:
            print(f"Warning: No matching breathing pattern for {visited_folder}")
            continue

        all_frames_path.append(visited_folder)

    return [all_frames_path, all_labels]

In [48]:
# get labels

all_rgb_frames_path = []
all_rgb_labels = []

for fps_bound in all_rgb_folders:
    tmp_path_list, tmp_labels_list = get_label_breathing(fps_bound)
    all_rgb_frames_path += tmp_path_list
    all_rgb_labels += tmp_labels_list

In [49]:
print(all_rgb_frames_path[0])
print(all_rgb_labels[0])

E:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun/2 Meters/With Blankets/Hold Breath/frames_Arun2_lower_bound
1


# Splitting the data into training and testing